## Use Case 3: Comparing BMI above and below 25 across the proteomics data

These are the tools we will use to play with the data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

This is the data we will be playing with

In [2]:
import CPTAC

Loading Clinical Data...
Loading Proteomics Data...
Loading Transcriptomics Data...
Loading CNA Data...
Loading Phosphoproteomics Data...
Loading Somatic Data...

 ******PLEASE READ******


Use case 3: BMI above and below 25
The first step is to load the clinical dataframe and the dataframe to compare it with

In [3]:
clinical = CPTAC.get_clinical()
proteomics = CPTAC.get_proteomics()

Next we will use the compare_clinical() function to create a dataframe that appends a column from the clinical dataframe to our chosen dataframe

In [4]:
bmiProt = CPTAC.compare_clinical(clinical, proteomics, 'BMI')
print(bmiProt)

        BMI  A1BG   A2M  A2ML1  A4GALT  AAAS  AACS  AADAT  AAED1  AAGAB  ...   \
idx                                                                      ...    
S001  38.88 -1.01 -0.81  -0.28    0.24  0.29  0.68   1.27  -0.33   0.27  ...    
S002  39.76 -0.51 -1.00  -0.99    1.50  0.18  0.37   1.26   0.15   1.34  ...    
S003  51.19 -0.56 -1.33   0.64     NaN -0.26  1.05  -0.10  -0.14   0.46  ...    
S004  32.69 -1.53 -1.19  -0.49    0.26 -0.03  0.77   1.10  -0.93   0.26  ...    
S005  20.28 -0.16  0.09   0.01    0.34  0.51 -0.05  -1.18    NaN   0.66  ...    
S006  55.67 -1.03 -0.63  -0.04   -0.25 -0.09 -0.11   0.85   1.27   0.17  ...    
S007  25.68 -1.09 -0.60  -1.11    0.02  0.16  0.67  -0.37  -0.12   0.33  ...    
S008  21.57 -0.29  0.51  -0.51     NaN  0.46  0.05  -0.22   0.18   0.11  ...    
S009  34.26 -0.93 -1.28   0.67    0.43 -0.05  0.86  -0.22    NaN  -0.21  ...    
S010  36.57 -0.44 -0.87   2.83   -0.32  0.18  0.47  -0.15  -0.83  -0.01  ...    
S011  27.83 -0.13  0.03   0.

Now we can split the bmiCNA dataframe into two dataframes based on whether the patient has a BMI (or whatever variable was specified in the previous step) above or below 25.
This is done by using the dataframe logic to create an array of boolean values, which we can then use to select the respective dataframes

In [5]:
greaterBool = bmiProt['BMI'] >= 25
lessBool = bmiProt['BMI'] < 25

highBMI = bmiProt[greaterBool]
lowBMI = bmiProt[lessBool]
print(highBMI)
print(lowBMI)

        BMI  A1BG   A2M  A2ML1  A4GALT  AAAS  AACS  AADAT  AAED1  AAGAB  ...   \
idx                                                                      ...    
S001  38.88 -1.01 -0.81  -0.28    0.24  0.29  0.68   1.27  -0.33   0.27  ...    
S002  39.76 -0.51 -1.00  -0.99    1.50  0.18  0.37   1.26   0.15   1.34  ...    
S003  51.19 -0.56 -1.33   0.64     NaN -0.26  1.05  -0.10  -0.14   0.46  ...    
S004  32.69 -1.53 -1.19  -0.49    0.26 -0.03  0.77   1.10  -0.93   0.26  ...    
S006  55.67 -1.03 -0.63  -0.04   -0.25 -0.09 -0.11   0.85   1.27   0.17  ...    
S007  25.68 -1.09 -0.60  -1.11    0.02  0.16  0.67  -0.37  -0.12   0.33  ...    
S009  34.26 -0.93 -1.28   0.67    0.43 -0.05  0.86  -0.22    NaN  -0.21  ...    
S010  36.57 -0.44 -0.87   2.83   -0.32  0.18  0.47  -0.15  -0.83  -0.01  ...    
S011  27.83 -0.13  0.03   0.61    2.03 -0.10  1.34    NaN  -0.87   0.26  ...    
S012  34.84  1.55  1.67   0.43    0.70 -0.56  1.14    NaN    NaN   0.31  ...    
S013  68.39 -0.03  0.14   2.

We can now check for genes that have a significantly different protein abundance between the high and low BMI. First we need to set a more accurate threshold since we have so many samples.

In [23]:
threshold = .05 / len(highBMI.columns) #TODO: this doesn't yield anything significant yet
print("Threshold:", threshold)
significantTests = []
significantGenes = []
for num in range(1,len(highBMI.columns)):
    gene = highBMI.columns[num]
    ttest = stats.ttest_ind(highBMI[gene], lowBMI[gene])
    if ttest.pvalue <= threshold:
        significantTests.append(ttest)
        significantGenes.append(gene)
print(len(significantGenes))
#largerMean = largerThan25.mean()
#smallerMean = smallerThan25.mean()
#print(largerMean)
#print(smallerMean)

Threshold: 5.215939912372209e-06


C:\Users\David\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\David\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\David\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


0
